In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import unicodedata
import pycountry

In [2]:
sample = pd.read_stata('../data/sample.dta')
sample.head()

,id,Ofirstnam,Omidnam,Osurname,msa
0,1.0,A,DALE,MAYO,9999.0
1,2.0,MARY,ANN,SKREZEC,7362.0
2,3.0,RAY,HOWARD,SUHLER,6450.0
3,4.0,VUONG,TUYEN,KHON,9999.0
4,5.0,SON,SUK,LEE,3920.0


## AAAI Conference

In [3]:
aaai_df = pd.read_csv('../data/aaai_authors.csv')
len(aaai_df)

# 2004
# 97
# change from 2002
# if ; in jj then:
# split by , and remove last as university

29349

In [4]:
aaai_df.head()

,year,paper_id,title,url,author_name,university,country
0,18,16583,Algorithms for Trip-Vehicle Assignment in Ride...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Xiaohui Bei,Nanyang Technological University,Singapore
1,18,16583,Algorithms for Trip-Vehicle Assignment in Ride...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Shengyu Zhang,<span>The Chinese University of Hong Kong</span>,Hong Kong
2,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Pin-Yu Chen,IBM Research AI,United States
3,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Yash Sharma,NaN,"The Cooper Union, New York"
4,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Huan Zhang,NaN,"University of California, Davis"


## Only keep from 2018 - 2005

In [8]:
aaai_df2 = pd.read_csv('../data/aaai_18_05.csv')
aaai_df2 = aaai_df2.where((pd.notnull(aaai_df2)), None)
total_aaai2 = len(aaai_df2)
aaai_df2.head()

,year,paper_id,title,url,author_name,university,country
0,18,16583,Algorithms for Trip-Vehicle Assignment in Ride...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Xiaohui Bei,Nanyang Technological University,Singapore
1,18,16583,Algorithms for Trip-Vehicle Assignment in Ride...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Shengyu Zhang,<span>The Chinese University of Hong Kong</span>,Hong Kong
2,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Pin-Yu Chen,IBM Research AI,United States
3,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Yash Sharma,None,"The Cooper Union, New York"
4,18,16893,EAD: Elastic-Net Attacks to Deep Neural Networ...,https://aaai.org/ocs/index.php/AAAI/AAAI18/pap...,Huan Zhang,None,"University of California, Davis"


In [37]:
def clean_chars(s):
    s_clean= ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
    return s_clean.upper()

def clean_and_name(name):
    if not name:
        return None
    name = clean_chars(name)
    if name.split(' ')[0] == 'AND':
        return ' '.join(name.split(' ')[1:])
    if name.split(' ')[-1] == 'AND':
        return ' '.join(name.split(' ')[:-1])
    else:
        return name

def split_name(name):
    name = clean_and_name(name)
    if not name:
        return pd.Series([None, None, None])
    first_name = None
    middle_name = None
    surname = None
    name_split = str(name).split(' ')
    if len(name_split) == 1:
        first_name  = name_split[0]
    elif len(name_split) == 2:
        first_name  = name_split[0]
        surname = name_split[1]
    elif len(name_split) == 3:
        first_name  = name_split[0]
        middle_name = name_split[1]
        surname = name_split[2]
    else:
        return pd.Series([None, None, None])
    if  len(re.sub('[.]', '', first_name)) == 1:
        first_name = None
    if middle_name: 
        if len(re.sub('[.]', '', middle_name)) == 1:
            middle_name = None
    if surname:
        if len(re.sub('[.]', '', surname)) == 1:
            surname = None
    return pd.Series([first_name, middle_name, surname])

def cleanhtml(raw_html):
    if raw_html:
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', raw_html)
        return cleantext

In [10]:
aaai_df2['first_name'] = None
aaai_df2['middle_name'] = None
aaai_df2['surname'] = None
aaai_df2[['first_name', 'middle_name', 'surname']] = aaai_df2['author_name'].apply(lambda x: split_name(x))
aaai_df2[['author_name', 'first_name', 'middle_name', 'surname', 'country']].head()

,author_name,first_name,middle_name,surname,country
0,Xiaohui Bei,XIAOHUI,None,BEI,Singapore
1,Shengyu Zhang,SHENGYU,None,ZHANG,Hong Kong
2,Pin-Yu Chen,PIN-YU,None,CHEN,United States
3,Yash Sharma,YASH,None,SHARMA,"The Cooper Union, New York"
4,Huan Zhang,HUAN,None,ZHANG,"University of California, Davis"


In [11]:
aaai_df2[['author_name', 'first_name', 'middle_name', 'surname', 'university','country']]

,author_name,first_name,middle_name,surname,university,country
0,Xiaohui Bei,XIAOHUI,None,BEI,Nanyang Technological University,Singapore
1,Shengyu Zhang,SHENGYU,None,ZHANG,<span>The Chinese University of Hong Kong</span>,Hong Kong
2,Pin-Yu Chen,PIN-YU,None,CHEN,IBM Research AI,United States
3,Yash Sharma,YASH,None,SHARMA,None,"The Cooper Union, New York"
4,Huan Zhang,HUAN,None,ZHANG,None,"University of California, Davis"
5,Jinfeng Yi,JINFENG,None,YI,None,Tencent AI Lab
6,Cho-Jui Hsieh,CHO-JUI,None,HSIEH,None,"University of California, Davis"
7,Jonathan Chung,JONATHAN,None,CHUNG,None,University of Toronto
8,Moshe Eizenman,MOSHE,None,EIZENMAN,None,University of Toronto
9,Uros Rakita,UROS,None,RAKITA,None,University of Toronto


In [78]:
#first_aaai = aaai_df2[['author_name', 'first_name', 'middle_name', 'surname', 'university', 'country']].copy()

#first_aaai.groupby(['author_name', 'first_name', 'middle_name', 
#                    'surname', 'country']).grouper.group_info[1]

649

In [126]:
first_aaai = aaai_df2[['author_name', 'first_name', 'middle_name', 'surname', 'university', 'country']].copy()

#first_aaai.groupby(['author_name', 'first_name', 'middle_name', 'surname', 'country']).grouper.group_info[0]
first_aaai = first_aaai.drop_duplicates()
unique_aaai = len(first_aaai)
first_aaai = first_aaai[~(pd.isnull(first_aaai['first_name']) | 
                                 (first_aaai['first_name'] == 'None'))].copy()
first_aaai = first_aaai.dropna(subset=['first_name'])
first_aaai.reset_index(inplace=True)
first_aaai.rename(columns={'index':'id', 
                           'first_name': 'Ofirstnam', 
                           'middle_name': 'Omidnam', 
                           'surname': 'Osurname'}, inplace=True)
first_aaai.dropna(subset=['Ofirstnam'])
#first_aaai['id'] = first_aaai['id'].apply(lambda x: 'aaai_1_{}'.format(x))
unique_clean_names = len(first_aaai)
#first_aaai.to_csv('../data/clean_name/aaai_first_18_05.csv', index=False)
print('papers {}'.format(total_aaai2))
print('unique {}'.format(unique_aaai))
print('unique clean {}'.format(unique_clean_names))
first_aaai.head()

papers 20233
unique 15263
unique clean 14633


,id,author_name,Ofirstnam,Omidnam,Osurname,university,country
0,0,Xiaohui Bei,XIAOHUI,None,BEI,Nanyang Technological University,Singapore
1,1,Shengyu Zhang,SHENGYU,None,ZHANG,<span>The Chinese University of Hong Kong</span>,Hong Kong
2,2,Pin-Yu Chen,PIN-YU,None,CHEN,IBM Research AI,United States
3,3,Yash Sharma,YASH,None,SHARMA,None,"The Cooper Union, New York"
4,4,Huan Zhang,HUAN,None,ZHANG,None,"University of California, Davis"


In [127]:
first_aaai.head()

,id,author_name,Ofirstnam,Omidnam,Osurname,university,country
0,0,Xiaohui Bei,XIAOHUI,None,BEI,Nanyang Technological University,Singapore
1,1,Shengyu Zhang,SHENGYU,None,ZHANG,<span>The Chinese University of Hong Kong</span>,Hong Kong
2,2,Pin-Yu Chen,PIN-YU,None,CHEN,IBM Research AI,United States
3,3,Yash Sharma,YASH,None,SHARMA,None,"The Cooper Union, New York"
4,4,Huan Zhang,HUAN,None,ZHANG,None,"University of California, Davis"


In [128]:
second_aaai = aaai_df2[pd.isnull(aaai_df2['first_name'])]

In [129]:
countries = pd.DataFrame([x.name for x in list(pycountry.countries)], columns=['country'])
countries['country_name'] = countries['country']
countries['country_code'] = [x.alpha_2 for x in list(pycountry.countries)]
countries.head(10)

,country,country_name,country_code
0,Aruba,Aruba,AW
1,Afghanistan,Afghanistan,AF
2,Angola,Angola,AO
3,Anguilla,Anguilla,AI
4,Åland Islands,Åland Islands,AX
5,Albania,Albania,AL
6,Andorra,Andorra,AD
7,United Arab Emirates,United Arab Emirates,AE
8,Argentina,Argentina,AR
9,Armenia,Armenia,AM


In [130]:
vars = ['id', 'author_name', 'Ofirstnam', 'Omidnam', 'Osurname', 'country_code']
#first_aaai.merge(countries, how='left', on='country')[vars].to_csv('../data/clean_name/aaai_first_18_05.csv', index=False)

In [131]:
first_aaai2 = first_aaai.merge(countries, how='left', on='country')

In [132]:
len(first_aaai)

14633

In [133]:
first_aaai2 = first_aaai2.where((pd.notnull(first_aaai2)), None)
first_aaai2['affiliation'] = first_aaai2.apply(lambda x: x['university'] if x['university']
                                              else x['country'] if not x['country_name']
                                              else None, axis=1)

In [134]:
first_aaai2['clean_affiliation'] = first_aaai2['affiliation'].map(cleanhtml)

In [135]:
first_aaai2

,id,author_name,Ofirstnam,Omidnam,Osurname,university,country,country_name,country_code,affiliation,clean_affiliation
0,0,Xiaohui Bei,XIAOHUI,None,BEI,Nanyang Technological University,Singapore,Singapore,SG,Nanyang Technological University,Nanyang Technological University
1,1,Shengyu Zhang,SHENGYU,None,ZHANG,<span>The Chinese University of Hong Kong</span>,Hong Kong,Hong Kong,HK,<span>The Chinese University of Hong Kong</span>,The Chinese University of Hong Kong
2,2,Pin-Yu Chen,PIN-YU,None,CHEN,IBM Research AI,United States,United States,US,IBM Research AI,IBM Research AI
3,3,Yash Sharma,YASH,None,SHARMA,None,"The Cooper Union, New York",None,None,"The Cooper Union, New York","The Cooper Union, New York"
4,4,Huan Zhang,HUAN,None,ZHANG,None,"University of California, Davis",None,None,"University of California, Davis","University of California, Davis"
5,5,Jinfeng Yi,JINFENG,None,YI,None,Tencent AI Lab,None,None,Tencent AI Lab,Tencent AI Lab
6,6,Cho-Jui Hsieh,CHO-JUI,None,HSIEH,None,"University of California, Davis",None,None,"University of California, Davis","University of California, Davis"
7,7,Jonathan Chung,JONATHAN,None,CHUNG,None,University of Toronto,None,None,University of Toronto,University of Toronto
8,8,Moshe Eizenman,MOSHE,None,EIZENMAN,None,University of Toronto,None,None,University of Toronto,University of Toronto
9,9,Uros Rakita,UROS,None,RAKITA,None,University of Toronto,None,None,University of Toronto,University of Toronto


In [144]:
# read old
first_old = pd.read_csv('../data/clean_name/aaai_first_18_05.csv')
first_old = first_old.where((pd.notnull(first_old)), None)

In [148]:
len(first_old)

14408

In [149]:
first_old = first_old.drop_duplicates(subset=['author_name','Ofirstnam','Omidnam','Osurnmame','country_code'])

In [150]:
first_aaai2.head()

,id,author_name,Ofirstnam,Omidnam,Osurname,university,country,country_name,country_code,affiliation,clean_affiliation
0,0,Xiaohui Bei,XIAOHUI,None,BEI,Nanyang Technological University,Singapore,Singapore,SG,Nanyang Technological University,Nanyang Technological University
1,1,Shengyu Zhang,SHENGYU,None,ZHANG,<span>The Chinese University of Hong Kong</span>,Hong Kong,Hong Kong,HK,<span>The Chinese University of Hong Kong</span>,The Chinese University of Hong Kong
2,2,Pin-Yu Chen,PIN-YU,None,CHEN,IBM Research AI,United States,United States,US,IBM Research AI,IBM Research AI
3,3,Yash Sharma,YASH,None,SHARMA,None,"The Cooper Union, New York",None,None,"The Cooper Union, New York","The Cooper Union, New York"
4,4,Huan Zhang,HUAN,None,ZHANG,None,"University of California, Davis",None,None,"University of California, Davis","University of California, Davis"


In [151]:
first_old.head()

,id,author_name,Ofirstnam,Omidnam,Osurname,country_code
0,aaai_1_0,Xiaohui Bei,XIAOHUI,None,BEI,SG
1,aaai_1_1,Shengyu Zhang,SHENGYU,None,ZHANG,HK
2,aaai_1_2,Pin-Yu Chen,PIN-YU,None,CHEN,US
3,aaai_1_3,Yash Sharma,YASH,None,SHARMA,None
4,aaai_1_4,Huan Zhang,HUAN,None,ZHANG,None


In [152]:
first_old[first_old['Ofirstnam'] == 'SASA']

,id,author_name,Ofirstnam,Omidnam,Osurname,country_code
5193,aaai_1_6281,Sasa Grbic,SASA,None,GRBIC,None
10373,aaai_1_14004,Saša Petrovic ́,SASA,PETROVIC,None,None


In [156]:
au = first_aaai2[['author_name','country_code','clean_affiliation']].merge(first_old, on=['author_name', 'country_code'], how='left')

In [154]:
len(au)

14633

In [158]:
au[['id','clean_affiliation']].to_csv('../data/clean_name/affiliations_aaai.csv', index=False)

In [92]:
len(first_old)

14408

In [93]:
len(first_aaai2)

14633